In [1]:
# default_exp core

In [2]:
# export
# Python native modules
import os,json
from typing import *
# Third party libs
from fastcore.all import *
import nbformat
# Local modules

IN_NOTEBOOK=False

In [3]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON


# CLI
> ghissue2docs primarly is just a cli. This will allow you to pull issues and generate either md, or jupternotebooks
for usage with nbdev or auto-documentation tools that injest mds.

Note for the below functions to run properly, you need to use the `gh auth login` command.

In [4]:
# export
def get_issues(label=None)->dict:
    json_cmd='--json assignees,author,body,title,url,id'
    if label is None: out=run(f'gh issue list {json_cmd}',stderr=True)  
    else:             out=run(f'gh issue list -l {label} {json_cmd}',stderr=True)
    return json.loads(out)

In [5]:
get_issues('documentation')

[{'assignees': [],
  'author': {'login': 'josiahls'},
  'body': 'This is an example documentation issue\r\n\r\nBelow is a picture!\r\n![image](https://user-images.githubusercontent.com/19930483/146814046-fc69bf3f-bc6a-41d4-9971-146b5884ee55.png)\r\n',
  'id': 'I_kwDOGj3YrM5ArPed',
  'title': 'Examples.Test Documentation Issue!',
  'url': 'https://github.com/josiahls/ghissue2docs/issues/1'}]

In [4]:
test_issue=[{'assignees': [],
  'author': {'login': 'josiahls'},
  'body': 'This is an example documentation issue\r\n\r\nBelow is a picture!\r\n![image](https://user-images.githubusercontent.com/19930483/146814046-fc69bf3f-bc6a-41d4-9971-146b5884ee55.png)\r\n',
  'id': 'I_kwDOGj3YrM5ArPed',
  'title': 'Examples.Test Documentation Issue!',
  'url': 'https://github.com/josiahls/ghissue2docs/issues/1'}]

test_issue_no_nest=[{'assignees': [],
  'author': {'login': 'josiahls'},
  'body': 'This is an example documentation issue\r\n\r\nBelow is a picture!\r\n![image](https://user-images.githubusercontent.com/19930483/146814046-fc69bf3f-bc6a-41d4-9971-146b5884ee55.png)\r\n',
  'id': 'I_kwDOGj3YrM5ArPed',
  'title': 'Test Documentation Issue!',
  'url': 'https://github.com/josiahls/ghissue2docs/issues/1'}]

In [5]:
# export
_image_not_loading="Oh No! You should not be seeing this! Make sure you're logged into github if the repo is private!"

def issue2txt(issue:Dict)->Tuple[List[str],List[str],str]:
    nests=issue['title'].split('.')[:-1]
    sections=[
f"""# {issue['title'].split('.')[-1]}\n""",
f"""Issue [#{issue['url'].split('/')[-1]}]({issue['url']})<br>
Author: @{issue['author']['login']}
<hr>
{issue['body'].replace('![image]','<br>!['+_image_not_loading+']')}
"""
]
    if IN_NOTEBOOK:
        from IPython.display import display, Markdown, Latex
        display(Markdown('<br>'.join(sections)))
    return sections,issue['id'],nests
    

In [6]:
issue2txt(test_issue_no_nest[0])

# Test Documentation Issue!
<br>Issue [#1](https://github.com/josiahls/ghissue2docs/issues/1)<br>
Author: @josiahls
<hr>
This is an example documentation issue

Below is a picture!
<br>![Oh No! You should not be seeing this! Make sure you're logged into github if the repo is private!](https://user-images.githubusercontent.com/19930483/146814046-fc69bf3f-bc6a-41d4-9971-146b5884ee55.png)



(['# Test Documentation Issue!\n',
  "Issue [#1](https://github.com/josiahls/ghissue2docs/issues/1)<br>\nAuthor: @josiahls\n<hr>\nThis is an example documentation issue\r\n\r\nBelow is a picture!\r\n<br>![Oh No! You should not be seeing this! Make sure you're logged into github if the repo is private!](https://user-images.githubusercontent.com/19930483/146814046-fc69bf3f-bc6a-41d4-9971-146b5884ee55.png)\r\n\n"],
 'I_kwDOGj3YrM5ArPed',
 [])

In [7]:
issue2txt(test_issue[0])

# Test Documentation Issue!
<br>Issue [#1](https://github.com/josiahls/ghissue2docs/issues/1)<br>
Author: @josiahls
<hr>
This is an example documentation issue

Below is a picture!
<br>![Oh No! You should not be seeing this! Make sure you're logged into github if the repo is private!](https://user-images.githubusercontent.com/19930483/146814046-fc69bf3f-bc6a-41d4-9971-146b5884ee55.png)



(['# Test Documentation Issue!\n',
  "Issue [#1](https://github.com/josiahls/ghissue2docs/issues/1)<br>\nAuthor: @josiahls\n<hr>\nThis is an example documentation issue\r\n\r\nBelow is a picture!\r\n<br>![Oh No! You should not be seeing this! Make sure you're logged into github if the repo is private!](https://user-images.githubusercontent.com/19930483/146814046-fc69bf3f-bc6a-41d4-9971-146b5884ee55.png)\r\n\n"],
 'I_kwDOGj3YrM5ArPed',
 ['Examples'])

In [8]:
# export
def issue2markdown(text:List[str],issueid:str,nesting:List[str]=None,to:Path=Path('.')):
    if nesting is not None:
        for n in nesting: to/=n
    to/=f'{issueid}.md'
    to.mk_write('\n'.join(text))
    return to

In [9]:
txt,issueid,nests=issue2txt(test_issue[0])

# Test Documentation Issue!
<br>Issue [#1](https://github.com/josiahls/ghissue2docs/issues/1)<br>
Author: @josiahls
<hr>
This is an example documentation issue

Below is a picture!
<br>![Oh No! You should not be seeing this! Make sure you're logged into github if the repo is private!](https://user-images.githubusercontent.com/19930483/146814046-fc69bf3f-bc6a-41d4-9971-146b5884ee55.png)



In [10]:
out_path=issue2markdown(txt,issueid,nests,Path('.'));out_path

Path('Examples/I_kwDOGj3YrM5ArPed.md')

In [11]:
print(f'Contents of {out_path}')
print(Path(out_path).read_text())

Contents of Examples/I_kwDOGj3YrM5ArPed.md
# Test Documentation Issue!

Issue [#1](https://github.com/josiahls/ghissue2docs/issues/1)<br>
Author: @josiahls
<hr>
This is an example documentation issue

Below is a picture!
<br>![Oh No! You should not be seeing this! Make sure you're logged into github if the repo is private!](https://user-images.githubusercontent.com/19930483/146814046-fc69bf3f-bc6a-41d4-9971-146b5884ee55.png)




In [12]:
# export
def issue2ipynb(text:List[str],issueid:str,nesting:List[str]=None,to:Path=Path('.')):
    if nesting is not None:
        for n in nesting: to/=n
    if nesting: to=f'{99}_{".".join(nesting)}.{issueid}.ipynb'
    else:       to=f'{99}_{issueid}.ipynb'
    
    nb=nbformat.v4.new_notebook()
    
    title_cell=nbformat.v4.new_markdown_cell(text[0])
    if len(text)<1: other_cells=[]  
    else:           other_cells=[nbformat.v4.new_markdown_cell(s) for s in text[1:]] 
    
    nb['cells']=[title_cell]+other_cells
    nbformat.write(nb,to)
    return to

In [13]:
out_path=issue2ipynb(txt,issueid,[],Path('.'));out_path

'99_I_kwDOGj3YrM5ArPed.ipynb'

In [14]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import make_readme
    make_readme()
    notebook2script()
    notebook2html()

converting /home/ghissue2docs_user/ghissue2docs/nbs/index.ipynb to README.md
Converted 00_core.ipynb.
Converted 99_Examples.I_kwDOGj3YrM5ArPed.ipynb.
Converted 99_I_kwDOGj3YrM5ArPed.ipynb.
Converted index.ipynb.
converting: /home/ghissue2docs_user/ghissue2docs/nbs/00_core.ipynb
converting: /home/ghissue2docs_user/ghissue2docs/nbs/99_I_kwDOGj3YrM5ArPed.ipynb
